# Fitting an early MLE
In this notebook we will fit an early MLE model to audiovisual speech perception data.

In [4]:
import numpy as np
from glob import glob

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from scipy.stats import norm
from scipy.optimize import minimize

from likelihoods import compute_log_likelihood

## Prepare Data
The data consists of five text files, each containing seven rows and five columns, where:
- Row 1: Audiotorial data
- Row 2: Visual data
- Rows 3-7: Audiovisual data
    - a combination of rows 1 and 2
    - visual goes from 'b' (row 3) to 'd' (row 7) 
    - audio goes from 'b' (col 1) to 'd' (col 5)

In [5]:
# get paths to data files
file_paths = glob("./data/*.txt")

# load all data into a single array
data = np.array([np.loadtxt(fname) for fname in file_paths])
N, M, K = data.shape

# define number of samples for each subject
n_samples = 24 

# split into the three different data types
# data_A = data[:, 0, :]
# data_V = data[:, 1, :]
# data_AV = data[:, 2:, :]

## Fit MLE

In [10]:
def compute_params(c_A, c_V, std_A, std_V):
    """Compute the parameters for the MLE model from the free parameters"""

    # get variances from standard deviations
    var_A = std_A ** 2
    var_V = std_V ** 2

    # compute denominator for calculations
    den = 1 / (var_A + var_V) 

    # compute mean and variance
    mu_AV = np.zeros((5, 5))
    for a in np.arange(5):
        for v in np.arange(5):
            mu_AV[a, v] = ((a+1 - c_A) * var_V + (v+1 - c_V) * var_A) * den
    var_AV = var_A * var_V * den

    # return mean and standard deviation
    return mu_AV, np.sqrt(var_AV)

In [11]:
def objective_function(theta, data, n_samples):
    """ Compute MLE objective function on data (7, 5) from a single subject
        where theta=[c_A, c_V, log(std_A), log(std_V)]."""

    # extract parameters
    c_A, c_V, std_A, std_V = theta
    std_A, std_V = np.exp(std_A), np.exp(std_V)

    # compute shifted means
    mu_At = c_A - (np.arange(5) + 1)
    mu_Vt = c_V - (np.arange(5) + 1)

    # compute parameters for audiovisual
    mu_AV, std_AV = compute_params(c_A, c_V, std_A, std_V)

    # compute the probabilities
    p_A = norm.cdf(data[0], mu_At, std_A)
    p_V = norm.cdf(data[1], mu_Vt, std_V)
    p_AV = norm.cdf(data[2:], mu_AV, std_AV)

    # compute and return the negative log-likelihood
    return compute_log_likelihood(p_A, p_V, p_AV, data, n_samples)

In [12]:
def mle_fit(data):
    """Perform MLE fit to data (7, 5) for a single subject"""
    theta = np.random.uniform(low=-1, high=1, size=4)
    opt_result = minimize(objective_function, theta, args=(data, n_samples))

    objective = opt_result.fun
    c_A, c_V, std_A, std_V = opt_result.x
    std_A, std_V = np.exp(std_A), np.exp(std_V)

    return objective, c_A, c_V, std_A, std_V

In [13]:
mle_fit(data[0])

c:\Users\sap98\Documents\GitHub\bayesian-audiovisual-speech-perception\likelihoods.py:12: RuntimeWarning: divide by zero encountered in log
  L = np.log(binomial_pmf(data, n_samples, probs)).sum()
c:\Users\sap98\Documents\GitHub\bayesian-audiovisual-speech-perception\venv\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


(inf,
 -0.8984219626048697,
 -0.6203280322490676,
 0.9713493539373578,
 2.0223504285663356)